# text2sql based on llama2
## Task: To get an sql query for a given database from a natural language query

Build a chain with:<br/> Question => LLM => SQL => DB => LLM => Answer

(This file is for experimentation and model creation)

### Imports

In [1]:
from langchain_community.llms.ollama import Ollama

from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.utilities.sql_database import SQLDatabase

from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.chains.sql_database import query

from langchain.agents import Tool
from langchain import agents

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 
from langchain_community.agent_toolkits.sql import base

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

from pswrd import PASSWORD_OF_DB
from pswrd import PASSWORD_FOR_VC_CREATOR

### Load llama2

In [2]:
model = Ollama(model="llama2:13b", temperature=0, verbose=True)

## The 1st idea:
Is add a **prompt template** with the context of the database structure and a query for NL to `llama2`

#### Firstly add DB structure

In [3]:
DB_STRUCTURE = \
    ["""
    CREATE TABLE trip(
        id BIGSERIAL PRIMARY KEY,
        company BIGINT, 
        plane CHARACTER VARYING(60),
        town_from CHARACTER VARYING(60),
        town_to CHARACTER VARYING(60),
        time_out TIMESTAMP,
        time_in TIMESTAMP,
        CONSTRAINT FK_company FOREIGNT KEY (\"company\") REFERENCES public.company (id)
    ); 
    """,
     """
    CREATE TABLE company(
        id BIGSERIAL PRIMARY KEY,
        name CHARACTER VARYING(60) 
    );
    """,
     """
    CREATE TABLE pass_in_trip(
        id BIGSERIAL PRIMARY KEY,
        trip BIGINT,
        passenger BIGINT,
        place CHARACTER VARYING(60),
        CONSTRAINT FK_trip FOREIGNT KEY (\"trip\") REFERENCES public.trip (id),
        CONSTRAINT FK_passanger FOREIGNT KEY (\"passenger\") REFERENCES public.passenger (id)
    );
    """,
     """
    CREATE TABLE passenger(
        id BIGSERIAL PRIMARY KEY,
        name CHARACTER VARYING(60)
    );
    """
    ]

In [4]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test",
    user="pgvc_embeddings_creator",
    password=PASSWORD_FOR_VC_CREATOR,
)
COLLECTION_NAME = "text2sql_vc"

In [3]:
embeddings = OllamaEmbeddings(model="llama2:13b")

In [6]:
structure_retriver = PGVector.from_texts(
    embedding=embeddings,
    texts=DB_STRUCTURE,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
    use_jsonb=True
).as_retriever()

#### Now create a template

In [7]:
template = \
"""
Translate the following query to sql using the following database structure. 
{structure}
As an answer, provide an sql query for postgresql.
Query to translate: {query}
"""

prompt_with_db_structure = PromptTemplate.from_template(template)

In [8]:
model_with_structure_context = (
    {"structure": structure_retriver, "query": RunnablePassthrough()}
    | prompt_with_db_structure
    | model
    | StrOutputParser()
)

In [9]:
res_query = model_with_structure_context.invoke("Select the names of all the people who are in the airline database")
res_query

' Sure! Here\'s the SQL query to translate the given query to PostgreSQL:\n\nSELECT passenger.name FROM passenger \nJOIN pass_in_trip ON passenger.id = pass_in_trip.passenger \nJOIN trip ON pass_in_trip.trip = trip.id;\n\nExplanation:\n\n1. We start by selecting all the rows from the "passenger" table.\n2. We then join the "passenger" table with the "pass_in_trip" table on the "id" column. This gives us all the rows where there is a matching row in the "pass_in_trip" table.\n3. Finally, we join the "pass_in_trip" table with the "trip" table on the "trip" column. This gives us all the rows where there is a matching row in both the "pass_in_trip" and "trip" tables.\n\nThe query above will return all the names of people who are in the airline database.'

In [10]:
print(res_query)

 Sure! Here's the SQL query to translate the given query to PostgreSQL:

SELECT passenger.name FROM passenger 
JOIN pass_in_trip ON passenger.id = pass_in_trip.passenger 
JOIN trip ON pass_in_trip.trip = trip.id;

Explanation:

1. We start by selecting all the rows from the "passenger" table.
2. We then join the "passenger" table with the "pass_in_trip" table on the "id" column. This gives us all the rows where there is a matching row in the "pass_in_trip" table.
3. Finally, we join the "pass_in_trip" table with the "trip" table on the "trip" column. This gives us all the rows where there is a matching row in both the "pass_in_trip" and "trip" tables.

The query above will return all the names of people who are in the airline database.


The answer looks like it's true, but it's not. The model is hallucinating

## The 2nd idea: Use Langchain sql query tamplate

### Create new connection, with readonly privileges

In [4]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test-2",
    user="postgres",
    password=PASSWORD_OF_DB,
)
COLLECTION_NAME = "table_with_data_to_read"

In [5]:
db = SQLDatabase.from_uri(CONNECTION_STRING)

In [13]:
db.run("SELECT * FROM public.\"passenger\"")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer')]"

In [14]:
print(db.table_info)


CREATE TABLE company (
	id BIGSERIAL NOT NULL, 
	name VARCHAR(60), 
	CONSTRAINT company_pkey PRIMARY KEY (id)
)

/*
3 rows from company table:
id	name
1	American Airlines
*/


CREATE TABLE pass_in_trip (
	id BIGSERIAL NOT NULL, 
	trip BIGINT, 
	passenger BIGINT, 
	place VARCHAR(60), 
	CONSTRAINT pass_in_trip_pkey PRIMARY KEY (id), 
	CONSTRAINT fk_passanger FOREIGN KEY(passenger) REFERENCES passenger (id), 
	CONSTRAINT fk_trip FOREIGN KEY(trip) REFERENCES trip (id)
)

/*
3 rows from pass_in_trip table:
id	trip	passenger	place
1	1	1	A11
*/


CREATE TABLE passenger (
	id BIGSERIAL NOT NULL, 
	name VARCHAR(60), 
	CONSTRAINT passenger_pkey PRIMARY KEY (id)
)

/*
3 rows from passenger table:
id	name
1	John
2	James
3	Poul
*/


CREATE TABLE passengers (
	id SERIAL NOT NULL, 
	name VARCHAR(50), 
	CONSTRAINT passengers_pkey PRIMARY KEY (id)
)

/*
3 rows from passengers table:
id	name

*/


CREATE TABLE trip (
	id BIGSERIAL NOT NULL, 
	company BIGINT, 
	plane VARCHAR(60), 
	town_from VARCHAR(60)

### Create seq2sql chain

In [15]:
sql_query_chain = query.create_sql_query_chain(model, db)

Check template in sql chain

In [16]:
print(sql_query_chain.get_prompts()[0].template)

You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to ru

In [17]:
res = sql_query_chain.invoke({"question": "Select the names of all the people who are in the airline database"})
print(res)

Question: Select the names of all the people who are in the airline database

SQLQuery: SELECT name FROM passenger;


The model is hallucinating and therefore gives an incorrect answer. It also ignores some of the requirements of the template.
<br/>Conclusion: **LLaMa2 is not up to the task due to the size**.

## The 3rd idea: Try to use few-shots training strategy
#### Write examples with current DB structure

In [6]:
examples = [
    {
        "input": "How many passengers are in the database?",
        "query": "SELECT COUNT(*) FROM public.\"passenger\";"
    },
    {
        "input": "What are the departure times of all flights?",
        "query": "SELECT \"time_out\" FROM public.\"trip\""
    },
    {
        "input": "What is Jane's place?",
        "query": "SELECT \"place\" FROM public.\"pass_in_trip\" JOIN public.\"passenger\" ON public.\"passenger\".\"id\" = public.\"pass_in_trip\".\"passenger\" WHERE public.\"passenger\".\"name\" = \'John\'"
    },
    {
        "input": "Give me all information about airlines",
        "query": "SELECT * FROM public.\"company\""
    },
    {
        "input": "Show me all the trips that are flying out today",
        "query": "SELECT * FROM public.\"trip\"\nWHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM \"time_out\")"
    },
    {
        "input": "Which planes depart from Washington?",
        "query": "SELECT \"plane\" FROM public.\"trip\" WHERE \"town_from\" = \'Washington\'"
    },
    {
        "input": "Print out the names of all the planes", 
        "query": "SELECT \"plane\" FROM public.\"trip\""
    },
    {
        "input": "How many people fly on Airbus?",
        "query": "SELECT COUNT(*) FROM public.\"pass_in_trip\" AS paip JOIN public.\"trip\" ON trip.\"id\" = paip.\"trip\" WHERE trip.\"plane\" = \'Airbus\'"
    },
]

#### Create a template
Prefix and suffix from sql query template

In [19]:
prefix = \
"""You are a PostreSQL expert. Given an input question, first create {top_k} syntactically correct PostreSQL query to run, then look at the results and take most correct.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Below are a number of examples of questions and their corresponding SQL queries."""

suffix = \
"""Only use the following tables:
{table_info}

Use the following format:

User input: {input}
SQL query: """

few_shots_prompt = PromptTemplate.from_template(
    "User input: {input}\nSQL query: {query}"
)

few_shots_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=few_shots_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "table_info", "top_k"],
)

The request turns out to be very large, I'm not sure what LLaMa can handle, but it's worth a try

In [20]:
few_shots_chain = query.create_sql_query_chain(llm=model, db=db, prompt=few_shots_prompt)

In [21]:
res = few_shots_chain.invoke({"question": "Select the names of all the people who are in the airline database"})
print(res)

User input: Select the names of all the people who are in the airline database

SQL query: SELECT name FROM passenger;

Explanation: This query retrieves the names of all the people in the "passenger" table. Since we want to retrieve all the names, we don't need to use any condition or join. We can simply select the "name" column from the "passenger" table.


The model does not understand what is wanted from her. I think this is due to a disproportionately large request for a relatively small model

### Let's try to add a model that will clean the query

In [22]:
template = "You're a typo and grammar filter. Here is the user\'s request. \
    Correct all the typos, but don\'t change the word order: {question}\
        Leave only the answer in the following format and don't write anything other than that:\nAnswer: answer here"
clear_question_prompt = PromptTemplate.from_template(template)

In [23]:
sql_query_chain_with_cleaning = (
    {"question": RunnablePassthrough()}
    | clear_question_prompt
    | model
    | StrOutputParser()
)

sql_query_chain_with_cleaning = (
    {"question": sql_query_chain_with_cleaning}
    | sql_query_chain
)

In [24]:
res = sql_query_chain_with_cleaning.invoke({"question": "Select the names of all the people who are in the airline database"})
print(res)

Question: Select the names of all the people who are in the airline database.

SQLQuery: SELECT name FROM passenger;


After 5 launches, 1 was even completely successful, but for the most part the model is still hallucinating

## The 4th idea: Use langchain agents and tools

In [25]:
sql_db_chain = query.create_sql_query_chain(model, db)

db_tool = Tool(
    name="airlines_db_agent",
    func=sql_db_chain.invoke,
    description="Use for extaract data from database"
)

template = """Answer the following questions as best you can. You have access to the following tools: {tools}
Use the following format:
Question: the input question you must answer Thought: you should always think about what to do Action: the action to take, should be one of [{tool_names}] Action Input: the input to the action Observation: the result of the action ... (this Thought/Action/Action Input/Observation can repeat N times) Thought: I now know the final answer Final Answer: the final answer to the original input question

Begin!
Question: {input} Thought:{agent_scratchpad}"""

prompt_with_tools = PromptTemplate.from_template(template)

db_agent = agents.create_react_agent(
    llm=model,
    tools=[db_tool],
    prompt=prompt_with_tools
)

In [26]:
res = agents.AgentExecutor\
    .from_agent_and_tools(db_agent, [db_tool])\
        .invoke({"input": "Select the names of all the people who are in the airline database"})
print(res)

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `
Thought: To select the names of all the people in the airline database, I can use the `airlines_db_agent` tool to query the database.

Action: Use `airlines_db_agent` to query the database and retrieve a list of all the names in the airline database.

Input: None`

In [27]:
print(res["output"])

TypeError: string indices must be integers, not 'str'

still hallucinating...

## The 5th idea: Use SQLDatabaseToolkit

In [28]:
toolkit = SQLDatabaseToolkit(llm=model, db=db)

db_agent_toolkit = base.create_sql_agent(
    llm=model,
    toolkit=toolkit,
    max_iterations=5
)

In [29]:
res = db_agent_toolkit.invoke({"input": "Select the names of all the passenger who are in the airline database"})
print(res)

{'input': 'Select the names of all the passenger who are in the airline database', 'output': 'Agent stopped due to iteration limit or time limit.'}


## The 6th idea: Use `SQLCoder`, model based on `llama`

In [7]:
sqlcoder_model = Ollama(model='sqlcoder:15b', temperature=0, verbose=True)

In [31]:
toolkit = SQLDatabaseToolkit(llm=model, db=db)

db_agent_toolkit = base.create_sql_agent(
    llm=model,
    toolkit=toolkit
)

In [32]:
res = db_agent_toolkit.invoke({"input": "Select the names of all the passenger who are in the airline database"})
print(res)

{'input': 'Select the names of all the passenger who are in the airline database', 'output': 'Agent stopped due to iteration limit or time limit.'}


## The 7th idea: Use 2 models, `llama2` for NL and `SQLCoder` (*based on llama*) for generate SQL

In [33]:
template =\
"""
Here is the SQL {table_info}, select all columns and types to them and tables. Give the answer in the format:
Table: Column - The data type of this column
"""

get_names_template = PromptTemplate.from_template(template)

In [34]:
get_names_chain = (
    {"table_info": RunnablePassthrough()}
    | get_names_template
    | model
    | StrOutputParser()
)

In [35]:
res = get_names_chain.invoke({"table_info": db.table_info})
print(res)

 Sure! Here is the information you requested:

Table: company
Column - The data type of this column
id - bigserial (primary key)
name - varchar(60)

Table: pass_in_trip
Column - The data type of this column
id - bigserial (primary key)
trip - bigint (foreign key referencing trip table)
passenger - bigint (foreign key referencing passenger table)
place - varchar(60)
fk_passanger - foreign key referencing passenger table
fk_trip - foreign key referencing trip table

Table: passenger
Column - The data type of this column
id - bigserial (primary key)
name - varchar(60)

Table: passengers
Column - The data type of this column
id - serial (primary key)
name - varchar(50)

Table: trip
Column - The data type of this column
id - bigserial (primary key)
company - bigint (foreign key referencing company table)
plane - varchar(60)
town_from - varchar(60)
town_to - varchar(60)
time_out - timestamptz without time zone
time_in - timestamptz without time zone
fk_company - foreign key referencing compa

In [36]:
template =\
"""
Here is the user's request: {input}. 
Think about what the user would like to receive. Then select the 5 most necessary column and table names that are needed in the query. 
Take the column and column names from {table_names}. 
In the response, leave only a paraphrased user request written in natural language
DONT WRITE SQL
Rephrase the user's request on natural language, but with the correct names:
"""

nl_query_template = PromptTemplate.from_template(template)

In [37]:
nl_query_chain = (
    {"table_names": get_names_chain, "input": RunnablePassthrough()}
    | nl_query_template
    | model
    | StrOutputParser()
)

In [38]:
res = nl_query_chain.invoke({
    "input": "Select the names of all the passenger who are in the airline database",
    "table_info": db.table_info})
print(res)

 Based on the provided SQL query, the user would like to retrieve the names of all the passengers who are in the airline database. The necessary column and table names that are needed in the query are:

* passenger (from the pass_in_trip table)
* name (from the passenger table)

The user's request can be rephrased in natural language as follows:

"Please provide a list of all the passengers who are in the airline database, along with their names."


In [39]:
sqlcoder_chain = (
    {"question": nl_query_chain}
    | query.create_sql_query_chain(llm=sqlcoder_model, db=db)
    | StrOutputParser()
)

In [40]:
res = sqlcoder_chain.invoke({"input": "Select the names of all the passenger who are in the airline database"})
print(res)

The query should return a list of passenger names from the passenger table. This list should be sorted alphabetically by name and should not contain any duplicates.
The query should also include a count of how many times each passenger has been seen on trips, which can be calculated using the COUNT function in SQL.
Finally, the query should only return results for trips that are still ongoing (i.e., time_out > CURRENT_DATE).
To accomplish this task, we need to join several tables together:
The passenger table needs to be joined with the pass_in_trip table because each passenger can appear in multiple trips.
The trip table needs to be joined with the company table because each trip is associated with a specific airline.
Finally, the query should filter out results for trips that are not ongoing by comparing time_out with CURRENT_DATE.
The final result should be sorted alphabetically by passenger name and then counted how many times each passenger has been seen on trips.

Here is the SQL

## The 8th idea: Use `SQLDatabaseToolkit` with 2 models

In [16]:
toolkit = SQLDatabaseToolkit(llm=sqlcoder_model, db=db)

sqlcoder_toolkit = base.create_sql_agent(
    llm=model,
    toolkit=toolkit
)

In [42]:
res = sqlcoder_toolkit.invoke({"input": "Select the name of all the passenger who are in the airline database"})
print(res)

{'input': 'Select the name of all the passenger who are in the airline database', 'output': 'Agent stopped due to iteration limit or time limit.'}


In [17]:
for chunk in sqlcoder_toolkit.stream("Select the name of all the passenger who are in the airline database"):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT name FROM passengers`
---
Tool Result: ``
---


KeyboardInterrupt: 

## The 9th idea: Give to agent example

In [8]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5433,
    database="llama-test",
    user="pgvc_embeddings_creator",
    password=PASSWORD_FOR_VC_CREATOR,
)
COLLECTION_NAME = "text2sql_vc"

In [9]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    PGVector,
    k=2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True    
);

c:\Projects\text2sql\venv\Lib\site-packages\langchain_community\vectorstores\pgvector.py:293: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


In [10]:
question = "Select the name of all the passenger who are in the airline database?"
selected_examples = example_selector.select_examples({"question": question})

In [11]:
selected_examples

[{'input': 'How many passengers are in the database?',
  'query': 'SELECT COUNT(*) FROM public."passenger";'},
 {'input': 'Show me all the trips that are flying out today',
  'query': 'SELECT * FROM public."trip"\nWHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")'}]

In [12]:
suffix =\
"""You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer."""

few_shots_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate(template="Question: {input}\n{query}", input_variables=["input", "query"]),
    suffix="Question: {input}",
    input_variables=["input"]
)

In [13]:
toolkit = SQLDatabaseToolkit(llm=sqlcoder_model, db=db)

sqlcoder_toolkit = base.create_sql_agent(
    llm=model,
    toolkit=toolkit
)

In [14]:
sqlcoder_chain = (
    {"input": RunnablePassthrough()}
    | few_shots_prompt
    | sqlcoder_toolkit
)

In [50]:
res = sqlcoder_chain.invoke("Select the name of all the passenger who are in the airline database")
print(res)

{'input': StringPromptValue(text='Question: Show me all the trips that are flying out today\nSELECT * FROM public."trip"\nWHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")\n\nQuestion: Print out the names of all the planes\nSELECT "plane" FROM public."trip"\n\nQuestion: Select the name of all the passenger who are in the airline database'), 'output': 'Agent stopped due to iteration limit or time limit.'}


In [51]:
print(res["input"].text)

Question: Show me all the trips that are flying out today
SELECT * FROM public."trip"
WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")

Question: Print out the names of all the planes
SELECT "plane" FROM public."trip"

Question: Select the name of all the passenger who are in the airline database


In [52]:
print(res["output"])

Agent stopped due to iteration limit or time limit.


In [15]:
for chunk in sqlcoder_chain.stream("Select the name of all the passenger who are in the airline database"):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" WHERE EXTRACT(DAY FROM NOW()) = EXTRACT(DAY FROM "time_out")`
---
Tool Result: ``
---
Calling Tool: `sql_db_query` with input `SELECT * FROM public."trip" W

KeyboardInterrupt: 